In [1]:
gapi_key = ''

In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
import os
import sys
import shutil
sys.path.insert(0, '../lib/')

from geotools import GeoParser
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm
tqdm_notebook().pandas()

HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [26]:
def get_hash(s):
    if s:
        return str("%016x" % hash(str(s)))
    else:
        raise ValueError('Wrong argument is passed.')
        
def clear_img_dataset(path='../data/img/'):
    path_train = path+'train'
    path_test = path+'test'
    if os.path.exists(path_train):
        shutil.rmtree(path_train)
    if os.path.exists(path_test):
        shutil.rmtree(path_test)

In [5]:
path_data = '../data/'
path_train_csv = os.path.join(path_data, 'train.csv')
path_test_csv = os.path.join(path_data, 'test.csv')

In [6]:
df_train = pd.read_csv(path_train_csv, sep=';')
print(f"Train shape: {df_train.shape}")
df_train.head()

Train shape: (875, 3)


,id,polygon,data_volume
0,1,"POLYGON ((59.91911 54.24099,59.91741 54.24320,...",4219.427
1,2,"POLYGON ((60.08978 55.04878,60.08896 55.04921,...",7324.976
2,3,"POLYGON ((60.87079 55.41289,60.86338 55.41722,...",21579.441
3,4,"POLYGON ((61.36703 55.14427,61.36541 55.14559,...",4098.549
4,5,"POLYGON ((61.36401 55.20983,61.37484 55.21551,...",10682.117


In [7]:
df_test = pd.read_csv(path_test_csv, sep=';')
print(f"Test shape: {df_test.shape}")
df_test.head()

Test shape: (236, 2)


,id,polygon
0,10,"POLYGON ((61.19189 54.18306,61.10879 54.21144,..."
1,17,"POLYGON ((58.98091 53.40790,58.98713 53.41196,..."
2,18,"POLYGON ((59.00441 53.44653,59.02322 53.46078,..."
3,19,"POLYGON ((59.03314 53.39044,59.09956 53.43713,..."
4,26,"POLYGON ((59.03963 53.33598,59.00855 53.34708,..."


In [8]:
assert df_train.id.duplicated().sum() == 0
assert df_test.id.duplicated().sum() == 0
print('В id нет дубликатов.')

В id нет дубликатов.


In [9]:
df_train['hash_name'] = df_train.polygon.progress_apply(get_hash)
df_test['hash_name'] = df_test.polygon.progress_apply(get_hash)

HBox(children=(IntProgress(value=0, max=875), HTML(value='')))

HBox(children=(IntProgress(value=0, max=236), HTML(value='')))

In [28]:
# The following list shows the approximate level of detail you can expect to see at each zoom level:
# 1: World
# 5: Landmass/continent
# 10: City
# 15: Streets
# 20: Buildings

im_size = "1024x1024"
im_zoom = "15"
dict_df = {"train":df_train, "test":df_test}
clear_img_dataset()

for key in ('train', 'test'):
    counter = 0
    for i, data in tqdm(dict_df.get(key).iterrows(), total=dict_df.get(key).shape[0]):
        counter += 1
        poly_str = data.polygon
        hash_str = data.hash_name
        path_postfix = counter // 10 + 1
        path_name = f"imset_{path_postfix}"
        fname = f"{hash_str}_{im_size}.jpeg"
        path_set = os.path.join(path_data, 'img', key, path_name)
        if not os.path.exists(path_set):
            os.makedirs(path_set)
        fpath = os.path.join(path_set, fname)
        if not os.path.exists(fpath):
            GeoParser.poly2jpeg(poly_str, im_size, im_zoom, gapi_key, fpath)


 35%|███▌      | 309/875 [01:14<02:17,  4.11it/s]


 71%|███████   | 617/875 [02:27<01:01,  4.21it/s]


 21%|██        | 50/236 [00:16<01:04,  2.87it/s]


100%|██████████| 236/236 [01:18<00:00,  2.76it/s]

In [32]:
dict_fpaths = {'train':[],'test':[]}
for key in ('train', 'test'):
    counter = 0
    for i, data in tqdm(dict_df.get(key).iterrows(), total=dict_df.get(key).shape[0]):
        counter += 1
        poly_str = data.polygon
        hash_str = data.hash_name
        path_postfix = counter // 10 + 1
        path_name = f"imset_{path_postfix}"
        fname = f"{hash_str}_{im_size}.jpeg"
        path_set = os.path.join(path_data, 'img', key, path_name)
        fpath = os.path.join(path_set, fname)
        dict_fpaths[key].append(fpath)


100%|██████████| 236/236 [00:00<00:00, 6211.37it/s]

In [37]:
df_train['fpath'] = dict_fpaths['train']
df_test['fpath'] = dict_fpaths['test']

In [38]:
df_test.to_pickle(path_data+'df_test_whash.pickle')
df_train.to_pickle(path_data+'df_train_whash.pickle')